# Processamento de linguagem natural com spaCy e NLTK

In [4]:
#!pip install spacy

In [5]:
import bs4 as bs
import urllib.request
import nltk
import spacy

## Marcação POS

- POS (part-of-speech) atribui para as palavras partes da fala, como substantivos, adjetivos e verbos.
- Importante para a detecção de entidades no texto, pois primeiro é necessário saber o que o texto contém

In [15]:
#!python -m spacy download pt_core_news_sm

In [14]:
pln = spacy.load('pt_core_news_sm')
pln

In [18]:
documento = pln('Estou aprendendo processamento de linguagem natural.')
type(documento)

spacy.tokens.doc.Doc

In [19]:
for token in documento:
    print(token.text, token.pos_)  # tipo de palavra

Estou AUX
aprendendo VERB
processamento NOUN
de ADP
linguagem NOUN
natural ADJ
. PUNCT


## Lematização e stemização

In [20]:
for token in documento:
    print(token.text, token.lemma_)  # radical da palavra

Estou Estou
aprendendo aprender
processamento processamento
de de
linguagem linguagem
natural natural
. .


In [23]:
doc = pln('encontrei encontraram encontrarão encontrariam cursando curso cursei')
[token.lemma_ for token in doc]

['encontrar',
 'encontrar',
 'encontrar',
 'encontrar',
 'cursar',
 'cursar',
 'cursar']